In [24]:
import numpy as np
# import units as u
import numpy as np
import pandas as pd

import rasterio
from rasterio.plot import show_hist, show
import geopandas as gpd
import rasterstats
from shapely.geometry import shape

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt

from icecream import ic

In [2]:
# shape file definining administrative borders 
states_path = "data/states/nga_admbnda_adm1_osgof_20161215.shp"
states = gpd.read_file(states_path)

In [25]:
states_m = states.to_crs('epsg:32632')
state_areas = pd.DataFrame(states_m.apply(lambda x: (x.admin1Name, shape(x["geometry"]).area ), axis=1).to_list(), columns=["state", "area (m2)"])
state_areas.head()

# TODO => save this in csv or pickle file..

# state_areas["area (m2)"].sum()/ 1e6 # ~ 923,768 km2

,state,area (m2)
0,Abia,4.858882e+09
1,Adamawa,3.792499e+10
2,Akwa Ibom,6.736774e+09
3,Anambra,4.807933e+09
4,Bauchi,4.849640e+10


In [6]:
# raster data sources => wind, temperature, solar flux 

data_paths = {
    "wind_speed": "data/Nigeria_MeanWindSpeed/NGA_wind-speed_100m.tif",
    "temperature": "data/Nigeria_AvgDailyTotals_GlobalSolarAtlas_GEOTIFF/TEMP.tif",
    "solar_flux": "data/Nigeria_AvgDailyTotals_GlobalSolarAtlas_GEOTIFF/DNI.tif",
}

In [8]:
def get_state_means(data_path, states=states):
    # read raster files..
    data = rasterio.open(data_path)
    data_array = data.read()[0]
    affine = data.transform

    # calculate zonal statistics
    averaged_data = rasterstats.zonal_stats(states, data_array, affine=affine, stats = ["mean"], geojson_out=True)

    state_wind_average = []
    for item in averaged_data:
        state_wind_average.append((item["properties"]["admin1Name"],
        item["properties"]["mean"]))

    df = pd.DataFrame(state_wind_average, columns=["state", "value"])
    return df


In [11]:
data_averages = {}
for name, path in data_paths.items():
    data_averages[name] = get_state_means(path, states=states)

# TODO => save this in csv or pickle file..

In [19]:
avg_vals = {n: i["value"] for n, i in data_averages.items()}

In [13]:
def calculate_power_panel(F_cur, T_a, w):
    """
    F_cur = _ # W/m^2 current solar flux normal to the panel 
    T_a = _ # K / ºC, air temperature the panel is exposed to 
    w = _ # m/s, wind speed panel is exposed to 
    => all of these are arrays 

    return power potential of a single panel in the area defined by the characteristics above
    
    """

    # values that change based on design! 
    D_f = 0.864 # derating factor, product of correction factors for additional processes affecting solar output, Table 5.2 
    E_panel = 0.18 # solar panel efficiency obtained under standard test conditions -- Ex 5.2
    A_panel = 1.5 # m^2, surface area of the pane -- Ex 5.2
    
    # 5.9, cell temperature, empirical so units do not eqate 
    T_c = T_a + 0.32 * (F_cur/(8.91 + 2*w))

    b_ref = 0.0025 # / K, temperature coefficient 
    T_th = 55 # K, threshold temeprature 
    T_ref = 298.15 # K, reference temperature 
    # 5.8, correction for cell temperature 
    C_temp = 1 - b_ref * np.maximum( np.minimum(T_c - T_ref, T_th ), 0 )

    # 5.7 actual AC power output from a solar panel at a given time 
    # P_ac = p_mpp_stc * C_temp * D_f / F_1000
    P_ac = F_cur * A_panel * E_panel * C_temp * D_f # units?? # should i be considering the rating of the panel as well? 

    return P_ac


In [28]:
power_potential_panel  = calculate_power_panel(F_cur=avg_vals["solar_flux"], T_a=avg_vals["temperature"], w=avg_vals["wind_speed"])

In [ ]:
# determine the areas of each state, and determine the power potential if covered with identical arrays of solar panels..

In [36]:
def calc_power_potential(A_spacing, single_panel_potential):
    A_panel = 1.5 # m2
    n_panels = A_spacing / A_panel 
    farm_potential = np.multiply(single_panel_potential,n_panels)
    return farm_potential


In [30]:
np.multiply([1,1], [1,1])

array([1, 1])

In [37]:
power_potential_panel_farm = calc_power_potential(A_spacing=state_areas["area (m2)"], single_panel_potential=power_potential_panel)

In [38]:
power_potential_panel_farm 

0     1.674030e+09
1     2.467824e+10
2     2.150261e+09
3     1.756312e+09
4     3.202387e+10
5     3.027399e+09
6     1.403391e+10
7     5.004597e+10
8     7.738524e+09
9     5.784258e+09
10    2.450236e+09
11    7.057417e+09
12    2.351553e+09
13    2.894707e+09
14    3.430001e+09
15    1.138463e+10
16    1.754544e+09
17    1.556858e+10
18    2.612702e+10
19    1.344372e+10
20    1.620028e+10
21    2.219001e+10
22    1.234157e+10
23    1.569353e+10
24    1.373153e+09
25    1.278543e+10
26    3.663305e+10
27    5.751923e+09
28    5.402241e+09
29    3.098395e+09
30    1.088379e+10
31    1.586237e+10
32    3.316598e+09
33    2.115899e+10
34    3.225694e+10
35    3.087748e+10
36    2.132024e+10
dtype: float64